In [1]:
from pydantic_ai import Agent, BinaryContent
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dotenv import load_dotenv
import os
import nest_asyncio
from IPython.display import display, Audio
import requests
from token_creator import get_creds
nest_asyncio.apply()
load_dotenv()
from cortana_agent import Cortana_agent
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
hf_token=os.getenv('hf_token')
composio_api_key=os.getenv('composio_key')
openai_api_key=os.getenv('openai_api_key')
pse=os.getenv('pse')
creds=get_creds()


Error refreshing token: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=705846470233-jjuk8on68a7cg9gmhl81r6ftg6mgj44c.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56219%2F&scope=https%3A%2F%2Fmail.google.com%2F+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcontacts+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Ftasks+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.retriever&state=b3xGj0LUrF6j2rM2VPvtFwXwmnxTOR&access_type=offline


In [2]:
api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'openai_api_key':openai_api_key,
    'composio_key':composio_api_key
}

In [3]:
cortana=Cortana_agent(api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error, set `COMPOSIO_LOGGING_LEVEL=debug`.
  warnings.warn(
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use

In [4]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write as write_wav
import io

In [5]:
def record_audio(duration=3, sample_rate=44100, channels=1):
    """
    Record audio for a specified duration and return it as bytes.
    
    Args:
        duration (float): Recording duration in seconds
        sample_rate (int): Sample rate in Hz
        channels (int): Number of audio channels (1 for mono, 2 for stereo)
    
    Returns:
        bytes: Audio data in WAV format as bytes
    """
    print(f"Recording for {duration} seconds...")
    
    # Record audio
    recording = sd.rec(
        int(duration * sample_rate),
        samplerate=sample_rate,
        channels=channels,
        dtype='float32'
    )
    sd.wait()  # Wait until recording is finished
    print("Recording finished!")
    
    # Convert to int16 format (standard for WAV files)
    recording = (recording * 32767).astype(np.int16)
    
    # Save to bytes buffer
    buffer = io.BytesIO()
    write_wav(buffer, sample_rate, recording)
    audio_bytes = buffer.getvalue()
    buffer.close()
    return audio_bytes

In [8]:
audio=record_audio()
res=cortana.chat(['what is this saying?',BinaryContent(data=audio, media_type='audio/wav')])

API_URL = "https://router.huggingface.co/fal-ai/fal-ai/kokoro/american-english"
headers = {
    "Authorization": f"Bearer {hf_token}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

audio = query({
    "text": res.voice_version})
# You can access the audio with IPython.display for example

audio=audio.get("audio").get('url')
audio=Audio(audio, autoplay=True)



Recording for 3 seconds...
Recording finished!


In [9]:
audio